In [2]:
import requests
import pandas as pd
import json

In [3]:
API_KEY = "667e94661abb533b7c59e065eed32b7a-e8470dd11ead1617b105cb8843647d6c"
ACCOUNT_ID = "101-001-25318071-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [6]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [12]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [17]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [18]:
response = session.get(url, params=None, data=None, headers=None)

In [19]:
response.status_code

200

In [20]:
data = response.json()

In [24]:
instruments_list = data['instruments']

In [25]:
len(instruments_list)

68

In [27]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [28]:
key_inst = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [30]:
instruments_dict = {}

for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_inst}

In [32]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [33]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [34]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
        )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else: 
            data = data['candles']
    return response.status_code, data

In [35]:
code, data = fetch_candles("EUR_USD", count=20)

In [36]:
code

200

In [37]:
len(data)

20

In [38]:
data

[{'complete': True,
  'volume': 4910,
  'time': '2023-08-08T08:00:00.000000000Z',
  'bid': {'o': '1.09780', 'h': '1.09867', 'l': '1.09742', 'c': '1.09847'},
  'mid': {'o': '1.09787', 'h': '1.09874', 'l': '1.09749', 'c': '1.09855'},
  'ask': {'o': '1.09794', 'h': '1.09881', 'l': '1.09756', 'c': '1.09863'}},
 {'complete': True,
  'volume': 5024,
  'time': '2023-08-08T09:00:00.000000000Z',
  'bid': {'o': '1.09847', 'h': '1.09857', 'l': '1.09663', 'c': '1.09700'},
  'mid': {'o': '1.09856', 'h': '1.09865', 'l': '1.09670', 'c': '1.09708'},
  'ask': {'o': '1.09864', 'h': '1.09873', 'l': '1.09678', 'c': '1.09717'}},
 {'complete': True,
  'volume': 4952,
  'time': '2023-08-08T10:00:00.000000000Z',
  'bid': {'o': '1.09699', 'h': '1.09712', 'l': '1.09512', 'c': '1.09529'},
  'mid': {'o': '1.09707', 'h': '1.09720', 'l': '1.09520', 'c': '1.09536'},
  'ask': {'o': '1.09715', 'h': '1.09728', 'l': '1.09527', 'c': '1.09544'}},
 {'complete': True,
  'volume': 5199,
  'time': '2023-08-08T11:00:00.0000000